# RVC AI Cover Maker (separate parts version)
Created by [ShiromiyaG](https://github.com/ShiromiyaG)
- Colab inspired on [AICoverGen](https://github.com/SociallyIneptWeeb/AICoverGen) by [SociallyIneptWeeb](https://github.com/SociallyIneptWeeb)
- Uses the [blaise-tk](https://github.com/blaise-tk) version of [RVC_CLI](https://github.com/blaise-tk/RVC_CLI)

In [ ]:
import os
import subprocess
from google.colab import drive
from pathlib import Path
import requests
from IPython.display import clear_output
import subprocess
import fileinput
import zipfile
import shutil

drive.mount('/content/drive')
Path('/content/drive/MyDrive/RVC AI Cover Maker Input').mkdir(parents=True, exist_ok=True)
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
os.chdir("/content")
# @title # Install
# @markdown ##### Yes, it takes time, **and it's not necessary, since the other cells install the dependencies too**
# @markdown ##### Leave the extra files enabled only if you are going to use ensemble on vocals
voc_extra_files = True #@param {type:"boolean"}
# @markdown ##### To find bugs, you have the option of not clearing the output **(Causes crashes)**
not_clean_outputs = False #@param {type:"boolean"}
!sudo apt install ffmpeg python3.10-venv
!pip install poetry pedalboard
!poetry config virtualenvs.create false
Path('/content/arquivos').mkdir(parents=True, exist_ok=True)

%cd python-audio-separator
!pip install "audio-separator[gpu]" pyrubberband yt-dlp[default] wget git+https://github.com/IAHispano/gdown
if not_clean_outputs != True:
  clear_output()
%cd ../

if voc_extra_files == True:
  !git clone https://github.com/ZFTurbo/Music-Source-Separation-Training.git
  !wget -P Music-Source-Separation-Training/models "https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt"
  !wget -P Music-Source-Separation-Training/models "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/model_bs_roformer_ep_317_sdr_12.9755.yaml"
  !rm Music-Source-Separation-Training/utils.py
  !rm Music-Source-Separation-Training/requirements.txt
  !rm Music-Source-Separation-Training/inference.py
  !wget -P Music-Source-Separation-Training "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/utils.py"
  !wget -P Music-Source-Separation-Training "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/requirements.txt"
  !wget -P Music-Source-Separation-Training "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/inference.py"
  !pip install -r Music-Source-Separation-Training/requirements.txt
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/spectograma.py" # Spectograma
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/karokee_4band_v2_sn.pth" # karokee
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/download_checks.json" # download_checks
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/reverbpedalboard.py" # reverb_pedalboard
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/mix.py" # mix
if not_clean_outputs != True:
  clear_output()
!git clone https://github.com/shirounanashi/OrpheusDL.git
%cd OrpheusDL
if not_clean_outputs != True:
  !poetry install --no-root -q > /dev/null 2>&1
  clear_output()
else:
  !poetry install --no-root
%cd ../
!git clone https://git.ovosimpatico.com/ovosimpatico/orpheusdl-deezer.git ./OrpheusDL/modules/deezer
!wget -P OrpheusDL/config "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/settings.json"
if not_clean_outputs != True:
  clear_output()
!git clone https://github.com/shirounanashi/RVC_CLI.git
%cd RVC_CLI
if not_clean_outputs != True:
  !poetry install --no-root > /dev/null 2>&1
  clear_output()
else:
  !poetry install --no-root
%cd ../
!wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/fcpe.pt"
!wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/hubert_base.pt"
!wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/rmvpe.pt"
#!pip --disable-pip-version-check install beautifulsoup4 ffmpeg ffmpeg-python==0.1.18 numpy==1.23.5 requests==2.31.0 tqdm faiss-cpu==1.7.3 librosa==0.9.1 pydub==0.25.1 pyworld==0.3.4 praat-parselmouth==0.4.2 resampy==0.4.2 scipy==1.11.1 sounddevice==0.4.6 soundfile==0.12.1 torchaudio==2.1.1 praat-parselmouth noisereduce fairseq numba onnxruntime onnxruntime_gpu==1.15.1 torch==2.1.1 torchcrepe==0.0.21 torchgen>=0.0.1 torch_directml torchvision==0.16.1 einops local-attention matplotlib==3.7.2 tensorboard ffmpy==0.3.1 tensorboardX edge-tts==6.1.9 pydantic fastapi uvicorn
!pip uninstall onnxruntime-gpu -y
!python -m pip -q install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
if not_clean_outputs != True:
  clear_output()

# Send audio file

In [ ]:
#@title ### Local file upload
from google.colab import files
import os
import fnmatch
from pathlib import Path
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if file.endswith(".mp3") or file.endswith(".flac") or file.endswith(".wav"):
                caminho_arquivo = os.path.join(root, file)
                arquivos.append(caminho_arquivo)
    return arquivos
uploaded = files.upload()
uploaded = procurar_arquivos("/content")
for arquivo in uploaded:
  !cp "{arquivo}" "/content/musicas/arquivos-originais"

In [ ]:
#@title ### File from Google Drive
import os
import fnmatch
from google.colab import drive
from pathlib import Path
import importlib
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
drive_path = "/content/drive/MyDrive/RVC AI Cover Maker Input" #@param {type:"string"}

def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if fnmatch.fnmatch(file, '*.mp3') or fnmatch.fnmatch(file, '*.flac') or fnmatch.fnmatch(file, '*.wav'):
                arquivos.append(os.path.join(root, file))
    return arquivos

def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

if pacote_instalado("audio-separator"):
  pass
else:
  print("Dependencies not installed, setting up the Drive now")
  drive.mount('/content/drive')

arquivos_encontrados = procurar_arquivos(drive_path)

for arquivo in arquivos_encontrados:
    !cp "{arquivo}" "/content/musicas/arquivos-originais"

In [ ]:
#@title ### Download file from YouTube or Deezer
from IPython.display import clear_output
from pathlib import Path
import importlib
from glob import glob
import os
import json
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
# @markdown #### Select one of the options to download a song
link_of_yt = "" #@param {type:"string"}
link_of_deezer = "" #@param {type:"string"}
# @markdown ###### If you are going to use Deezer to download
bf_secret = "" #@param {type:"string"}
track_url_key = "" #@param {type:"string"}
arl = "" #@param {type:"string"}
input_folder = '/content/musicas/arquivos-originais'

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

if pacote_instalado("poetry"):
  pass
else:
  print("Dependencies not installed, installing now")
  !pip install yt-dlp[default] poetry
  !sudo apt install ffmpeg python3.10-venv
  !poetry config virtualenvs.create false
  clear_output()
  !git clone https://github.com/shirounanashi/OrpheusDL.git
  %cd OrpheusDL
  !poetry install --no-root -q > /dev/null 2>&1
  %cd ../
  clear_output()
  !git clone https://git.ovosimpatico.com/ovosimpatico/orpheusdl-deezer.git ./OrpheusDL/modules/deezer
  !wget -P OrpheusDL/config "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/settings.json"
  clear_output()

if link_of_deezer != "":
  with open('/content/OrpheusDL/config/settings.json', 'r') as file:
    data = json.load(file)

  data['modules']['deezer']['bf_secret'] = bf_secret
  data['modules']['deezer']['track_url_key'] = track_url_key
  data['modules']['deezer']['arl'] = arl

  with open('/content/OrpheusDL/config/settings.json', 'w') as file:
    json.dump(data, file, indent=4)

if link_of_yt != "":
  !yt-dlp --extract-audio -o "/content/musicas/arquivos-originais/%(title)s.%(ext)s"  --audio-format mp3 --audio-quality 0 "{link_of_yt}"
  clear_output()

if link_of_deezer != "":
  if bf_secret != "" and track_url_key != "" and arl != "":
    %cd OrpheusDL
    !python orpheus.py "{link_of_deezer}"
    clear_output()
    %cd ../
  else:
    raise ValueError("Você precisa fornecer a bf secret, track url key arl da sua conta Deezer Premium")

#Programs

In [ ]:
#@title ### Remove Backing Vocals and Echo/Reverb
from google.colab import drive
from IPython.display import Audio, display, clear_output
import os
def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

if pacote_instalado("onnx2torch"):
  pass
else:
  print("Dependencies not installed, installing now")
  !pip install "audio-separator[gpu]"
  drive.mount('/content/drive')
  Path(output_drive).mkdir(parents=True, exist_ok=True)
  clear_output()

from IPython.display import Audio, display, clear_output
import importlib
from glob import glob
from pydub import AudioSegment
from google.colab import drive
import importlib
input_folder = "/content/musicas/arquivos-originais"
no_back_folder = "/content/musicas/sem-back"
no_inst_folder = "/content/musicas/sem-intrumental"
output_folder = "/content/musicas/output-folder"
output_drive = "/content/drive/MyDrive/RVC AI Cover Maker"

def verificar_pasta_vazia(caminho_pasta):
    if not os.listdir(caminho_pasta):
        raise ValueError("You haven't sent an audio file!")

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    audio.export(output_file, format="mp3")

if os.path.exists("/content/arquivos/karokee_4band_v2_sn.pth"):
   pass
else:
  Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
  !wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/karokee_4band_v2_sn.pth"
  clear_output()

try:
    verificar_pasta_vazia(input_folder)
except ValueError as e:
    print(e)

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processando", len(input_files), "arquivos")
  if input_file.endswith(".mp3"):
    flac_filename = os.path.splitext(input_file)[0] + '.flac'
    if not os.path.exists(flac_filename):
        audio = AudioSegment.from_mp3(input_file)
        audio.export(f"/content/musicas/arquivos-originais/{flac_filename}", format="flac")
        os.remove(input_file)
        no_inst_output = flac_filename
  subprocess.run([
      'audio-separator',
      f"{no_inst_output}",
      '--model_file_dir', "/content/arquivos",
      '-m', 'karokee_4band_v2_sn.pth',
      '--vr_window_size', '320',
      '--vr_aggression', '5',
      "--normalization", "1.0",
      "--vr_batch_size", "8",
      '--vr_enable_tta',
      "--output_format", "FLAC",
      "--single_stem", "Vocals",
      '--output_dir', f"{no_back_folder}"
  ])
  filename = get_last_modified_file(no_back_folder)
  no_back_output = os.path.join(no_back_folder, filename)
  print(f"{filename} processing with karokee_4band_v2_sn is over!")
  subprocess.run([
      'audio-separator',
      f"{no_back_output}",
      '--model_file_dir', "/content/arquivos",
      '-m', 'UVR-DeEcho-DeReverb.pth',
      '--vr_window_size', '320',
      '--vr_aggression', '5',
      "--normalization", "1.0",
      "--vr_batch_size", "8",
      '--vr_enable_tta',
      "--output_format", "FLAC",
      "--single_stem", "No Reverb",
      '--output_dir', f"{output_folder}"
  ])
  processed_file = get_last_modified_file(output_folder)
  name = os.path.commonprefix(processed_file)
  !cp "{processed_file}" "{output_drive}"
  print(f"{filename} processing with UVR-DeEcho-DeReverb is over!")
  clear_output()
  print("Vocal processing completed.")
  print("The FLAC file is available in \"RVC AI Cover Maker\" folder on your Drive!")

In [ ]:
#@title ### Remove Instrumentals, Backing Vocals and Echo/Reverb
# @markdown ##### Merge Spectrograms
from google.colab import drive
import importlib
from pathlib import Path
from IPython.display import Audio, display, clear_output
import os
import fnmatch
output_drive = "/content/drive/MyDrive/RVC AI Cover Maker"
ensemble_voc = True #@param {type:"boolean"}
algorithm_ensemble_vocals =  "averege" # @param ['averege', 'Max Spec', 'Min Spec'] {allow-input: false}

def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

if pacote_instalado("audio-separator"):
  pass
else:
  print("Dependencias não instaladas, instalando agora")
  !pip install "audio-separator[gpu]" pydub
  if enbemble_voc == True:
    !git clone https://github.com/ZFTurbo/Music-Source-Separation-Training.git
    !wget -P Music-Source-Separation-Training/models "https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt"
    !wget -P Music-Source-Separation-Training/models "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/model_bs_roformer_ep_317_sdr_12.9755.yaml"
    !rm Music-Source-Separation-Training/utils.py
    !rm Music-Source-Separation-Training/requirements.txt
    !rm Music-Source-Separation-Training/inference.py
    !wget -P Music-Source-Separation-Training "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/utils.py"
    !wget -P Music-Source-Separation-Training "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/requirements.txt"
    !wget -P Music-Source-Separation-Training "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/inference.py"
    !pip install -r Music-Source-Separation-Training/requirements.txt
    !pip install pyrubberband
    !wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/spectograma.py"
  drive.mount('/content/drive')
  Path(output_drive).mkdir(parents=True, exist_ok=True)
  clear_output()

if os.path.exists("/content/arquivos/karokee_4band_v2_sn.pth"):
   pass
else:
  Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
  !wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/karokee_4band_v2_sn.pth"
  !wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/download_checks.json"
  clear_output()

from IPython.display import Audio, display, clear_output
from glob import glob
from pydub import AudioSegment
import subprocess
input_folder = "/content/musicas/arquivos-originais"
no_back_folder = "/content/musicas/sem-back"
no_inst_folder = "/content/musicas/sem-intrumental"
output_folder = "/content/musicas/output-folder"

Path(no_back_folder).mkdir(parents=True, exist_ok=True)
Path(no_inst_folder).mkdir(parents=True, exist_ok=True)
Path(output_folder).mkdir(parents=True, exist_ok=True)

def verificar_pasta_vazia(caminho_pasta):
    if not os.listdir(caminho_pasta):
        raise ValueError("You haven't sent an audio file!")

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if fnmatch.fnmatch(file, '*.mp3') or fnmatch.fnmatch(file, '*.flac') or fnmatch.fnmatch(file, '*.wav'):
                arquivos.append(os.path.join(root, file))
    return arquivos

try:
    verificar_pasta_vazia(input_folder)
except ValueError as e:
    print(e)

input_files = procurar_arquivos(input_folder)
for input_file in input_files:
  lista = []
  if input_file.endswith(".mp3"):
    flac_filename = os.path.splitext(input_file)[0] + '.flac'
    if not os.path.exists(flac_filename):
        audio = AudioSegment.from_mp3(input_file)
        audio.export(f"/content/musicas/arquivos-originais/{flac_filename}", format="flac")
        os.remove(input_file)
        input_file = flac_filename
  subprocess.run([
      'audio-separator',
      f"{input_file}",
      '--model_file_dir', "/content/arquivos",
      '-m', 'MDX23C-8KFFT-InstVoc_HQ.ckpt',
      "--normalization", "1.0",
      "--output_format", "FLAC",
      "--single_stem", "Vocals",
      "--mdxc_batch_size", "8",
      '--output_dir', f"{no_inst_folder}"
  ])
  if ensemble_voc == True:
    lista.append(get_last_modified_file(no_inst_folder))
    subprocess.run([
        "python",
        "Music-Source-Separation-Training/inference.py",
        "--model_type", "bs_roformer",
        "--config_path", "Music-Source-Separation-Training/models/model_bs_roformer_ep_317_sdr_12.9755.yaml",
        "--start_check_point", "Music-Source-Separation-Training/models/model_bs_roformer_ep_317_sdr_12.9755.ckpt",
        "--input_file", f"{input_file}",
        "--store_dir", f"{no_inst_folder}"
    ])
    lista.append(get_last_modified_file(no_inst_folder, "Vocals"))
    prefixo_comum = os.path.commonprefix(lista)
    ensemble_voc = os.path.join(no_inst_folder, f"{prefixo_comum}_ensemble1.flac")
    subprocess.run([
        "python", "/content/arquivos/spectograma.py", "--audio_input", lista[0], lista[1],
        "--algorithm", f"{algorithm_ensemble_vocals}",
        "--is_normalization", "False",
        "--wav_type_set", "PCM_16",
        "--save_path", f"{ensemble_voc}"
    ])
  filename = get_last_modified_file(no_inst_folder)
  no_inst_output = os.path.join(no_inst_folder, filename)
  print(f"{filename} processing with MDX23C-8KFFT-InstVoc_HQ is over!")
  subprocess.run([
      'audio-separator',
      f"{no_inst_output}",
      '--model_file_dir', "/content/arquivos",
      '-m', 'karokee_4band_v2_sn.pth',
      '--vr_window_size', '320',
      '--vr_aggression', '5',
      "--normalization", "1.0",
      "--vr_batch_size", "8",
      '--vr_enable_tta',
      "--output_format", "FLAC",
      "--single_stem", "Vocals",
      '--output_dir', f"{no_back_folder}"
  ])
  filename = get_last_modified_file(no_back_folder)
  no_back_output = os.path.join(no_back_folder, filename)
  print(f"{filename} processing with karokee_4band_v2_sn is over!")
  subprocess.run([
      'audio-separator',
      f"{no_back_output}",
      '--model_file_dir', "/content/arquivos",
      '-m', 'UVR-DeEcho-DeReverb.pth',
      '--vr_window_size', '320',
      '--vr_aggression', '5',
      "--normalization", "1.0",
      "--vr_batch_size", "8",
      '--vr_enable_tta',
      "--output_format", "FLAC",
      "--single_stem", "No Reverb",
      '--output_dir', f"{output_folder}"
  ])
  print(f"{filename} processing with UVR-DeEcho-DeReverb is over!")
  processed_file = get_last_modified_file(output_folder)
  !cp "{processed_file}" "{output_drive}"
clear_output()
print("Vocal processing completed.")
print("The FLAC file is available in \"RVC AI Cover Maker\" folder on your Drive!")

In [ ]:
#@title ### Remove Vocals (Keeping Backing Vocals)
from google.colab import drive
import os
import importlib
import subprocess
def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

if pacote_instalado("onnx2torch"):
  pass
else:
  !pip install "audio-separator[gpu]" "pydub" > /dev/null 2>&1
  clear_output()

from pydub import AudioSegment

input_folder = "/content/musicas/arquivos-originais"
stage1_dir = "/content/musicas/inst-etapa1"
stage2_dir = "/content/musicas/inst-etapa2"
final_output_dir = "/content/musicas/output_inst"
output_drive = "/content/drive/MyDrive/RVC AI Cover Maker"

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

if pacote_instalado("audio-separator"):
  pass
else:
  print("Dependencies not installed, installing now")
  !pip install "audio-separator[gpu]"
  drive.mount('/content/drive')
  Path(output_drive).mkdir(parents=True, exist_ok=True)
  clear_output()

if os.path.exists("/content/arquivos/spectograma.py"):
   pass
else:
  Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
  !wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/spectograma.py"
  !wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/karokee_4band_v2_sn.pth"
  clear_output()

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))

for input_file in input_files:
  print("Processando", len(input_files), "arquivos")
  ensemble1_inputs = []

  # Pass 1
  input_file = input_file
  if filename.endswith(".mp3"):
    flac_filename = os.path.splitext(input_file)[0] + '.flac'
    if not os.path.exists(flac_filename):
        audio = AudioSegment.from_mp3(input_file)
        audio.export(f"/content/musicas/arquivos-originais/{flac_filename}", format="flac")
        os.remove(input_file)
        input_file = flac_filename
    file_path = os.path.join(input_folder, input_file)
  basename, _ = os.path.splitext(input_file)
  pass1_outputs = []
  processed_models = []
  model_names = ["5_HP-Karaoke-UVR.pth", "UVR-MDX-NET-Inst_HQ_4.onnx", "htdemucs.yaml"]
  for model_name in model_names:
      if model_name == "5_HP-Karaoke-UVR.pth":
          subprocess.run([
              "audio-separator",
              f"{file_path}",
              "--model_file_dir", f"{model_dir}",
              "-m", f"{model_name}",
              "--normalization", "1.0",
              "--vr_batch_size", "8",
              "--vr_enable_tta",
              "--output_format", "FLAC",
              "--single_stem", "Instrumental",
              "--output_dir", f"{stage1_dir}"
          ])
          processed_models.append(model_name)
          print(f"{filename} processing with {model_name} is over!")
      if model_name == "UVR-MDX-NET-Inst_HQ_4.onnx":
          subprocess.run([
            "audio-separator",
            f"{file_path}",
            "--model_file_dir", f"{model_dir}",
            "-m", f"{model_name}",
            "--normalization", "1.0",
            "--mdx_batch_size", "8",
            "--output_format", "FLAC",
            "--single_stem", "Instrumental",
            "--output_dir", f"{stage1_dir}"
          ])
          processed_models.append(model_name)
          print(f"{filename} processing with {model_name} is over!")
      if model_name == "htdemucs.yaml":
          subprocess.run([
              "audio-separator",
              f"{file_path}",
              "--model_file_dir", f"{model_dir}",
              "-m", f"{model_name}",
              "--output_format", "FLAC",
              "--output_dir", f"{stage1_dir}"
          ])
          audio_files = [
              os.path.join(stage1_dir, f"{basename}_(Drums)_htdemucs.flac"),
              os.path.join(stage1_dir, f"{basename}_(Bass)_htdemucs.flac"),
              os.path.join(stage1_dir, f"{basename}_(Other)_htdemucs.flac")
          ]

          combined_audio = AudioSegment.from_file(audio_files[0], format="flac")

          for audio_file in audio_files[1:]:
              audio = AudioSegment.from_file(audio_file, format="flac")
              combined_audio = combined_audio.overlay(audio)

          combined_audio.export(f"{stage1_dir}/{basename}_demucs_(Instrumental).flac", format="flac")

          for audio_file in audio_files:
              os.remove(audio_file)
          processed_models.append(model_name)
          print(f"{filename} processing with {model_name} is over!")
      model_names = [model for model in model_names if model not in processed_models]


  all_files = os.listdir(stage1_dir)
  pass1_outputs_filtered = [os.path.join(stage1_dir, output) for output in all_files if "Instrumental" in output]
  name = os.path.commonprefix(pass1_outputs_filtered)
  # First Ensemble
  ensemble1_output = os.path.join(stage1_dir, f"{name}_ensemble1.flac")
  result = subprocess.run([
      "python", "/content/arquivos/spectograma.py", "--audio_input", pass1_outputs_filtered[0], pass1_outputs_filtered[1], pass1_outputs_filtered[2],
      "--algorithm", f"{algorithm_ensemble_inst}",
      "--is_normalization", "False",
      "--wav_type_set", "PCM_16",
      "--save_path", f"{ensemble1_output}"
  ])

  print("Processing of the first Ensemble is over!")

  # Pass 2
  basename = os.path.splitext(os.path.basename(ensemble1_output))[0]
  processed_models = []
  pass2_outputs = []
  model_names = ["karokee_4band_v2_sn.pth", "UVR-MDX-NET-Inst_HQ_4.onnx", "Kim_Vocal_2.onnx"]
  for model_name in model_names:
      if model_name == "karokee_4band_v2_sn.pth":
        model_name_without_ext = model_name.split('.')[0]
        output_path = os.path.join(stage2_dir, f"{basename}_(Instrumental)_{model_name_without_ext}.flac")
        pass2_outputs.append(output_path)
        subprocess.run([
            "audio-separator",
            f"{ensemble1_output}",
            "--model_file_dir", f"{model_dir}",
            "-m", model_name,
            "--vr_batch_size", "8",
            "--vr_enable_tta",
            "--normalization", "1.0",
            "--single_stem", "Instrumental",
            "--output_dir", f"{stage2_dir}"
        ])
        processed_models.append(model_name)
        print(f"Processamento de {basename} com o {model_name} acabou!")
      else:
        model_name_without_ext = model_name.split('.')[0]
        output_path = os.path.join(stage2_dir, f"{basename}_(Instrumental)_{model_name_without_ext}.flac")
        pass2_outputs.append(output_path)
        subprocess.run([
            "audio-separator",
            f"{ensemble1_output}",
            "--model_file_dir", f"{model_dir}",
            "-m", model_name,
            "--mdx_batch_size", "8",
            "--normalization", "1.0",
            "--single_stem", "Instrumental",
            "--output_dir", f"{stage2_dir}"
        ])
        processed_models.append(model_name)
        print(f"Processamento de {basename} com o {model_name} acabou!")
      model_names = [model for model in model_names if model not in processed_models]

  # Second Ensemble
  all_files = os.listdir(stage2_dir)
  pass2_outputs_filtered = [os.path.join(stage2_dir, output) for output in all_files if "Instrumental" in output]
  final_output_path = os.path.join(final_output_dir, f"{basename}_final_output.flac")
  subprocess.run([
      "python", "/content/arquivos/spectograma.py", "--audio_input", pass2_outputs_filtered[0], pass2_outputs_filtered[1], pass2_outputs_filtered[2],
      "--algorithm", f"{algorithm_ensemble_inst}",
      "--is_normalization", "False",
      "--wav_type_set", "PCM_16",
      "--save_path", f"{final_output_path}"
  ])
  print("Processing of the second Ensemble is over!")
  processed_file = get_last_modified_file(final_output_path)
  name = os.path.commonprefix(processed_file)
  !cp "{processed_file}" "{output_drive}"
  clear_output()
  print("Instrumental processing completed.")
  print("The FLAC file is available in \"RVC AI Cover Maker\" folder on your Drive!")

In [ ]:
#@title ### Apply Reverb
from google.colab import drive
import importlib
from glob import glob
def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

if pacote_instalado("pydub"):
  pass
else:
  drive.mount('/content/drive')
  !pip install pydub > /dev/null 2>&1
from IPython.display import Audio, display, clear_output
from pydub import AudioSegment
REVERB_SIZE = 0.15  # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_WETNESS = 0.2 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_DRYNESS = 0.8 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_DAMPING = 0.7 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
input_folder = "/content/musicas/arquivos-originais"
output_drive = "/content/drive/MyDrive/RVC AI Cover Maker"

def verificar_pasta_vazia(caminho_pasta):
    if not os.listdir(caminho_pasta):
        raise ValueError("You haven't sent an audio file!")

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

if os.path.exists("/content/arquivos/reverbpedalboard.py"):
   pass
else:
  Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
  !wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/reverbpedalboard.py"
  drive.mount('/content/drive')
  Path(output_drive).mkdir(parents=True, exist_ok=True)
  clear_output()

input_song = get_last_modified_file(input_folder)

try:
    verificar_pasta_vazia(input_song)
except ValueError as e:
    print(e)
input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processando", len(input_files), "arquivos")
  !python /content/arquivos/reverbpedalboard.py -i "{input_song}" -rsize "{REVERB_SIZE}" -rwet "{REVERB_WETNESS}" -rdry "{REVERB_DRYNESS}" -rdamp "{REVERB_DAMPING}" -oformat "WAV"
  song = f"{input_song}_moxed.wav"
  !cp song output_drive
  !cp "{processed_file}" "{output_drive}"
  clear_output()
  print("The FLAC file is available in \"RVC AI Cover Maker\" folder on your Drive!")

In [ ]:
#@title ### Remove RVC noise
from google.colab import drive
import importlib
from glob import glob
def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

if pacote_instalado("pydub"):
  pass
else:
  drive.mount('/content/drive')
  !pip install pydub > /dev/null 2>&1


from pydub import AudioSegment
input_folder = "/content/musicas/arquivos-originais"
noise_db_limit = -30 # @param {type:"slider", min:-50, max:0, step:0.1}
input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processando", len(input_files), "arquivos")
  input_song = input_file
  basename = os.path.basename(input_song)
  output_drive = f"/content/drive/MyDrive/RVC AI Cover Maker/{basename}_no_noise.wav"
  audio = AudioSegment.from_file(input_song)
  db_limit = noise_db_limit

  silenced_audio = AudioSegment.silent(duration=len(audio))
  chunk_length = 100
  for i in range(0, len(audio), chunk_length):
      chunk = audio[i:i+chunk_length]
      if chunk.dBFS > db_limit:
          silenced_audio = silenced_audio.overlay(chunk, position=i)

  silenced_audio.export(output_drive, format="wav")
  clear_output()
  print("The FLAC file is available in \"RVC AI Cover Maker\" folder on your Drive!")

In [ ]:
#@title ### Ensemble

import os
from pathlib import Path
from IPython.display import Audio, display, clear_output
path_to_input_folder = "/content/drive/MyDrive/input_musics" #@param {type:"string"}
path_to_output_folder = "/content/drive/MyDrive/output_ensemble" #@param {type:"string"}
output_drive = "/content/drive/MyDrive/RVC AI Cover Maker"
algoritmo = "averege" # @param ['averege', 'Max Spec', 'Min Spec'] {allow-input: false}

def verificar_pasta_vazia(caminho_pasta):
    if not os.listdir(caminho_pasta):
        raise ValueError("You haven't sent an audio file!")

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

if os.path.exists("/content/arquivos/spectograma.py"):
   pass
else:
  Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
  !wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/spectograma.py"
  drive.mount('/content/drive')
  clear_output()

Path(path_to_input_folder).mkdir(parents=True, exist_ok=True)
Path(path_to_output_folder).mkdir(parents=True, exist_ok=True)
files = [file for file in os.listdir(path_to_input_folder) if os.path.isfile(os.path.join(path_to_input_folder, file))]
common_name = os.path.commonprefix(files)

try:
    verificar_pasta_vazia(path_to_input_folder)
except ValueError as e:
    print(e)

save = "{path_to_output_folder}/{common_name}_ensemble_output.flac"

!python /content/arquivos/spectograma.py --audio_input {' '.join(files)} --algorithm algoritmo --is_normalization True --wav_type_set PCM_16 --save_path save
name = os.path.commonprefix(save)
!cp save output_drive
convert_to_mp3(save, name)
clear_output()
print("Instrumental processing completed.")
print("The FLAC file is available in \"RVC AI Cover Maker\" folder on your Drive!")
print("And MP3 in the player below!\n")
display(Audio(name, autoplay = False))

In [ ]:
#@title ### RVC inference
import importlib
from google.colab import drive
from pathlib import Path
from IPython.display import Audio, display, clear_output
import requests
import zipfile
from glob import glob
from pydub import AudioSegment
!pip install git+https://github.com/IAHispano/gdown > /dev/null 2>&1
rvc_model_link = ""  # @param {type:"string"}
rvc_model_name = os.path.splitext(os.path.basename(rvc_model_link))
f0method = "rmvpe"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
Pitch = 0  # @param {type:"slider", min:-24, max:24, step:0}
filter_radius = 3  # @param {type:"slider", min:0, max:10, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
autotune = False  # @param{type:"boolean"}
output_drive = "/content/drive/MyDrive/RVC AI Cover Maker"
model_destination_folder = f"/content/RVC_CLI/logs"
path_to_input_folder = "/content/drive/MyDrive/input_musics"

def verificar_pasta_vazia(caminho_pasta):
    if not os.listdir(caminho_pasta):
        raise ValueError("You haven't sent an audio file!")

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

def find_files(directory, extensions):
  files = glob(f'{directory}/**/*{extensions}', recursive=True)
  return files[0]

if os.path.exists("/content/RVC_CLI"):
   pass
else:
  Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
  !git clone https://github.com/shirounanashi/RVC_CLI.git
  %cd RVC_CLI
  !poetry install --no-root > /dev/null 2>&1
  clear_output()
  %cd ../
  !wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/fcpe.pt"
  !wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/hubert_base.pt"
  !wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/rmvpe.pt"
  drive.mount('/content/drive')
  Path(output_drive).mkdir(parents=True, exist_ok=True)
  clear_output()

if rvc_model_link == "":
  raise ValueError("You didn't put a link to an RVC model")

try:
    verificar_pasta_vazia(path_to_input_folder)
except ValueError as e:
    print(e)

print("Downloading model...")
filename = rvc_model_name
download_path = Path(model_destination_folder) / filename
if "drive.google.com" in rvc_model_link:
    gdown.download(rvc_model_link, str(download_path), quiet=False)
else:
    response = requests.get(rvc_model_link)
    with open(download_path, 'wb') as file:
        file.write(response.content)
if str(download_path).endswith(".zip"):
  Path(f'/content/RVC_CLI/logs/{rvc_model_name}').mkdir(parents=True, exist_ok=True)
  with zipfile.ZipFile(download_path, 'r') as zip_ref:
      zip_ref.extractall(f"/content/RVC_CLI/logs/{rvc_model_name}")
print("Download complete.")
current_dir = "/content/RVC_CLI"
model_folder = os.path.join(current_dir, f"logs/{rvc_model_name}")

if not os.path.exists(model_folder):
  raise FileNotFoundError(f"Model directory not found: {model_folder}")

files_in_folder = os.listdir(f"{model_destination_folder}/{rvc_model_name}")
pth_file = find_files(model_destination_folder, ".pth")
index_file = find_files(model_destination_folder, ".index")

if pth_file is None or index_file is None:
  raise FileNotFoundError("No model found.")
input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processando", len(input_files), "arquivos")
  output_path = "/content/output_rvc.flac"
  export_format = "FLAC"
  %cd RVC_CLI
  !python main.py infer --f0up_key "{Pitch}" --filter_radius "{filter_radius}" --index_rate "{index_rate}" --hop_length "{hop_length}" --rms_mix_rate "{rms_mix_rate}" --protect "{protect}" --f0autotune "{autotune}" --f0method "{f0method}" --input_path "{input_file}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "FLAC"
  %cd ../
  output_path = output_path.replace(".flac", f".{export_format.lower()}")
  !cp "{output_path}" "{output_drive}"
  clear_output()
  print("The FLAC file is available in \"RVC AI Cover Maker\" folder on your Drive!")